In [ ]:
import os
import numpy as np
import SimpleITK as sitk
from pathlib import Path

# Mapping des labels FreeSurfer
LABELS_MAP = {
    0: "Background", 2: "Left-Cerebral-White-Matter", 3: "Left-Cerebral-Cortex",
    4: "Left-Lateral-Ventricle", 5: "Left-Inf-Lat-Vent", 7: "Left-Cerebellum-White-Matter",
    8: "Left-Cerebellum-Cortex", 10: "Left-Thalamus-Proper", 11: "Left-Caudate",
    12: "Left-Putamen", 13: "Left-Pallidum", 14: "3rd-Ventricle", 15: "4th-Ventricle",
    16: "Brain-Stem", 17: "Left-Hippocampus", 18: "Left-Amygdala", 24: "CSF",
    26: "Left-Accumbens-area", 28: "Left-VentralDC", 30: "Left-vessel", 31: "Left-choroid-plexus",
    41: "Right-Cerebral-White-Matter", 42: "Right-Cerebral-Cortex", 43: "Right-Lateral-Ventricle",
    44: "Right-Inf-Lat-Vent", 46: "Right-Cerebellum-White-Matter", 47: "Right-Cerebellum-Cortex",
    49: "Right-Thalamus-Proper", 50: "Right-Caudate", 51: "Right-Putamen", 52: "Right-Pallidum",
    53: "Right-Hippocampus", 54: "Right-Amygdala", 58: "Right-Accumbens-area",
    60: "Right-VentralDC", 62: "Right-vessel", 63: "Right-choroid-plexus",
    72: "5th-Ventricle", 77: "WM-hypointensities", 80: "non-WM-hypointensities", 85: "Optic-Chiasm",
    251: "CC_Posterior", 252: "CC_Mid_Posterior", 253: "CC_Central", 254: "CC_Mid_Anterior", 255: "CC_Anterior",
}

# --- Setup des chemins ---
PATIENT, TP = "P1", "T1"
MODALITIES = ["T1", "T2", "FLAIR"]

base_path = Path(f"/Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/27919209/MSLesSegDataset/train/{PATIENT}/{TP}")
proj_path = Path("/Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/SIR-project")

aseg_path = proj_path / "data/07_registered_aseg_results" / f"{PATIENT}_{TP}_aseg.nii"
out_dir = proj_path / "data/16_template_txt"
out_dir.mkdir(parents=True, exist_ok=True)

def compute_2d_stats(img_slice, mask_slice, l_id):
    """Calcule la morphologie et le contraste sur la coupe cible."""
    l_zone = (mask_slice == l_id).astype(np.uint8)
    if not np.any(l_zone): return None

    # Intensités
    vals = img_slice[l_zone == 1]
    stats = {"mean": vals.mean(), "min": vals.min(), "max": vals.max()}
    
    # Géométrie via SITK
    img_sitk = sitk.GetImageFromArray(l_zone)
    shape = sitk.LabelShapeStatisticsImageFilter()
    shape.Execute(img_sitk)
    
    if shape.HasLabel(1):
        dims = shape.GetEquivalentEllipsoidDiameter(1)
        stats["major"], stats["minor"] = max(dims), min(dims)
    else: return None

    # Contraste simple (moyenne lésion / moyenne reste de la coupe)
    bg_mean = img_slice[l_zone == 0].mean()
    stats["contrast"] = stats["mean"] / bg_mean if bg_mean > 0 else 1.0
    return stats

def process_patient():
    mask_file = base_path / f"{PATIENT}_{TP}_MASK.nii.gz"
    if not mask_file.exists():
        print(f"Error: Mask missing -> {mask_file}")
        return

    # Chargement volumes
    m_img = sitk.ReadImage(str(mask_file))
    m_labels = sitk.RelabelComponent(sitk.ConnectedComponent(m_img > 0))
    m_arr = sitk.GetArrayFromImage(m_labels)
    n_lesions = int(m_arr.max())
    
    aseg_arr = sitk.GetArrayFromImage(sitk.ReadImage(str(aseg_path))) if aseg_path.exists() else None
    if aseg_arr is None: print("Warning: No ASEG found, location will be empty.")

    print(f"Processing {PATIENT} - {TP}: {n_lesions} lesions found.")

    for l_id in range(1, n_lesions + 1):
        # On cherche la slice avec le plus de pixels de la lésion l_id
        l_mask = (m_arr == l_id)
        best_z = np.argmax(l_mask.sum(axis=(1, 2)))
        
        data = {}
        for mod in MODALITIES:
            p = base_path / f"{PATIENT}_{TP}_{mod}.nii.gz"
            if p.exists():
                img = sitk.GetArrayFromImage(sitk.ReadImage(str(p)))
                res = compute_2d_stats(img[best_z], m_arr[best_z], l_id)
                if res: data[mod] = res
            
        if not data: continue

        # On prend FLAIR par défaut pour la morpho
        ref = data.get("FLAIR", list(data.values())[0])
        
        # Identification anatomique
        loc_str = "N/A"
        if aseg_arr is not None:
            l_pixels = aseg_arr[best_z][m_arr[best_z] == l_id]
            ids, counts = np.unique(l_pixels, return_counts=True)
            loc_list = []
            for i, c in zip(ids, counts):
                name = LABELS_MAP.get(int(i), f"ID-{int(i)}")
                loc_list.append(f"{name} ({100*c/len(l_pixels):.1f}%)")
            loc_str = " | ".join(loc_list)

        # Build du rapport texte
        report = [
            f"ID: {PATIENT}_{TP}_L{l_id}",
            f"Slice index (axial): {best_z}",
            "-"*30,
            f"Geometry: {ref['major']:.2f} x {ref['minor']:.2f} mm",
            "Intensities & Contrast:"
        ]
        for mod, s in data.items():
            report.append(f"  {mod:5}: Moy={s['mean']:.1f}, Contrast={s['contrast']:.2f}")
        
        report.append(f"\nLocation:\n  {loc_str}")

        out_name = f"{PATIENT}_{TP}_L{l_id}_desc.txt"
        with open(out_dir / out_name, "w", encoding="utf-8") as f:
            f.write("\n".join(report))
            
    print(f"Reports saved in {out_dir}")

if __name__ == "__main__":
    process_patient()
    
    # Check rapide du dernier fichier
    print("\n--- Quick Check ---")
    txt_files = list(out_dir.glob("*.txt"))
    if txt_files:
        with open(txt_files[-1], "r") as f:
            print(f.read())

--- Démarrage de l'analyse : P1 T1 ---
✅ Fichier ASEG trouvé : P1_T1_aseg.nii
✅ Extraction terminée. 18 fichiers générés dans /Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/SIR-project/data/16_template_txt

--- TEST DE VÉRIFICATION ---
Lecture du fichier de test : P1_T1_L17_2DDescription.txt

RAPPORT MÉDICAL: Patient P1 | T1 | Lésion 17
COUPE AXIALE: 94
MORPHOLOGIE (2D):
 - Grand axe: 5.47 mm
 - Petit axe: 4.66 mm

SIGNAL & CONTRASTE:
 - T1: Moy=173.6, Contrast=2.69
 - T2: Moy=216.4, Contrast=1.56
 - FLAIR: Moy=169.9, Contrast=2.60

LOCALISATION ANATOMIQUE:
 - Right-Cerebral-White-Matter (100.0%)

🎉 SUCCÈS : La localisation anatomique est bien présente !


In [13]:
import os
PROJET_ROOT = "/Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/SIR-project"
ASEG_DIR = os.path.join(PROJET_ROOT, "data", "07_registered_aseg_results")

if os.path.exists(ASEG_DIR):
    files = os.listdir(ASEG_DIR)
    print(f"✅ Dossier trouvé. Il contient {len(files)} fichiers.")
    print("Voici les 10 premiers fichiers :")
    for f in files[:10]:
        print(f" - {f}")
else:
    print(f"❌ LE DOSSIER N'EXISTE PAS : {ASEG_DIR}")

❌ LE DOSSIER N'EXISTE PAS : /Users/ilhanghauri/Documents/INSA/COURS/TC4/SIR/SIR-project/data/07_registered_aseg_results
